In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from tokenizers import ByteLevelBPETokenizer
import os

tokenizer_folder = "TokRoBERTa_BPE"
if os.path.isdir(tokenizer_folder) == False:
    os.mkdir(tokenizer_folder)

tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=["hi_dedup_1000.txt"], vocab_size=32000, min_frequency=3, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [5]:
tokenizer.save_model(tokenizer_folder)

['TokRoBERTa_BPE/vocab.json', 'TokRoBERTa_BPE/merges.txt']

In [6]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./TokRoBERTa_BPE/vocab.json",
    "./TokRoBERTa_BPE/merges.txt"
)

In [7]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [8]:
tokenizer.encode("महाप्रबंधक भारत संचार निगम लिमिटेड दुर्ग को सम्पत्तिकर")

Encoding(num_tokens=41, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [9]:
tokenizer.encode("महाप्रबंधक भारत संचार निगम लिमिटेड दुर्ग को सम्पत्तिकर").tokens

['<s>',
 'à¤®à¤¹',
 'à¤¾',
 'à¤ª',
 'à¥į',
 'à¤°à¤¬',
 'à¤Ĥ',
 'à¤§à¤ķ',
 'Ġà¤Ń',
 'à¤¾',
 'à¤°à¤¤',
 'Ġà¤¸',
 'à¤Ĥ',
 'à¤ļ',
 'à¤¾',
 'à¤°',
 'Ġà¤¨',
 'à¤¿',
 'à¤Ĺà¤®',
 'Ġà¤²',
 'à¤¿',
 'à¤®',
 'à¤¿',
 'à¤Ł',
 'à¥ĩ',
 'à¤¡',
 'Ġà¤¦',
 'à¥ģ',
 'à¤°',
 'à¥į',
 'à¤Ĺ',
 'Ġà¤ķ',
 'à¥ĭ',
 'Ġà¤¸à¤®',
 'à¥į',
 'à¤ªà¤¤',
 'à¥į',
 'à¤¤',
 'à¤¿',
 'à¤ķà¤°',
 '</s>']

In [10]:
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
TRAIN_EPOCHS = 15
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0.01
SEED = 42
MAX_LEN = 128
SUMMARY_LEN = 7

In [11]:
import torch

In [12]:
from transformers import RobertaConfig

config = RobertaConfig(
    # vocab_size=32000,
    # max_position_embeddings=514,
    # num_attention_heads=12,
    # num_hidden_layers=6,
    # type_vocab_size=1,
    attention_probs_dropout_prob=0.1,
    bos_token_id=0,
    eos_token_id=2,
    gradient_checkpointing=False,
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    hidden_size=768,
    initializer_range=0.02,
    intermediate_size=3072,
    layer_norm_eps=1e-05,
    max_position_embeddings=514,
    model_type="roberta",
    num_attention_heads=12,
    num_hidden_layers=12,
    pad_token_id=1,
    position_embedding_type="absolute",
    transformers_version="4.4.0.dev0",
    type_vocab_size=1,
    use_cache=True,
    vocab_size=32000
)

In [13]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("./TokRoBERTa_BPE", max_len=512)

In [14]:
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [15]:
model.num_parameters()

110651648

In [16]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./hi_dedup_1000.txt",
    block_size=128,
)

/Users/dishantpadalia/opt/miniconda3/envs/trocr/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [17]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [18]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./TokRoBERTa_BPE",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [19]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/Users/dishantpadalia/opt/miniconda3/envs/trocr/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 16
  Number of trainable parameters = 110651648


  0%|          | 0/16 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 218.4866, 'train_samples_per_second': 4.577, 'train_steps_per_second': 0.073, 'train_loss': 8.77268123626709, 'epoch': 1.0}


TrainOutput(global_step=16, training_loss=8.77268123626709, metrics={'train_runtime': 218.4866, 'train_samples_per_second': 4.577, 'train_steps_per_second': 0.073, 'train_loss': 8.77268123626709, 'epoch': 1.0})

In [20]:
trainer.save_model("./TokRoBERTa_BPE/")

Saving model checkpoint to ./TokRoBERTa_BPE/
Configuration saved in ./TokRoBERTa_BPE/config.json
Model weights saved in ./TokRoBERTa_BPE/pytorch_model.bin


In [21]:
trainer.push_to_hub()

OSError: Tried to clone a repository in a non-empty folder that isn't a git repository. If you really want to do this, do it manually:
git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.

In [38]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./TokRoBERTa_BPE/",
    tokenizer="./TokRoBERTa_BPE/"
)

loading configuration file ./TokRoBERTa_BPE/config.json
Model config RobertaConfig {
  "_name_or_path": "./TokRoBERTa_BPE/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}

loading configuration file ./TokRoBERTa_BPE/config.json
Model config RobertaConfig {
  "_name_or_path": "./TokRoBERTa_BPE/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_p

In [39]:
fill_mask("रहा है वह न सिर्फ चिंताजनक <mask>")

[{'score': 0.017935452982783318,
  'token': 264,
  'token_str': 'ा',
  'sequence': 'रहा है वह न सिर्फ चिंताजनका'},
 {'score': 0.005482870154082775,
  'token': 267,
  'token_str': '्',
  'sequence': 'रहा है वह न सिर्फ चिंताजनक्'},
 {'score': 0.004985986743122339,
  'token': 265,
  'token_str': 'े',
  'sequence': 'रहा है वह न सिर्फ चिंताजनके'},
 {'score': 0.002476259833201766,
  'token': 270,
  'token_str': 'ि',
  'sequence': 'रहा है वह न सिर्फ चिंताजनकि'},
 {'score': 0.0016431399853900075,
  'token': 269,
  'token_str': 'ी',
  'sequence': 'रहा है वह न सिर्फ चिंताजनकी'}]

In [40]:
fill_mask("पहलू सामने आ <mask> । अक्सर कंपनियों")

[{'score': 0.01747768558561802,
  'token': 264,
  'token_str': 'ा',
  'sequence': 'पहलू सामने आा । अक्सर कंपनियों'},
 {'score': 0.005214308854192495,
  'token': 267,
  'token_str': '्',
  'sequence': 'पहलू सामने आ् । अक्सर कंपनियों'},
 {'score': 0.005012022331357002,
  'token': 265,
  'token_str': 'े',
  'sequence': 'पहलू सामने आे । अक्सर कंपनियों'},
 {'score': 0.0025437825825065374,
  'token': 270,
  'token_str': 'ि',
  'sequence': 'पहलू सामने आि । अक्सर कंपनियों'},
 {'score': 0.0017022050451487303,
  'token': 269,
  'token_str': 'ी',
  'sequence': 'पहलू सामने आी । अक्सर कंपनियों'}]

In [41]:
fill_mask("मैं उनसे नफरत करती हूँ जिन्होंने किसी को अच्छे काम करने से रोका है. मैं नहीं जानती कि इसमें कितना समय और लगेगा, और मैं यह भी नहीं जानती कि मेरी मां घर वापस कब आएँगी, मुझे डाटेंगी, मुझे खाना खाने को कहेंगी, या मुझे <mask>")

[{'score': 0.018545446917414665,
  'token': 264,
  'token_str': 'ा',
  'sequence': 'मैं उनसे नफरत करती हूँ जिन्होंने किसी को अच्छे काम करने से रोका है. मैं नहीं जानती कि इसमें कितना समय और लगेगा, और मैं यह भी नहीं जानती कि मेरी मां घर वापस कब आएँगी, मुझे डाटेंगी, मुझे खाना खाने को कहेंगी, या मुझेा'},
 {'score': 0.00518343411386013,
  'token': 267,
  'token_str': '्',
  'sequence': 'मैं उनसे नफरत करती हूँ जिन्होंने किसी को अच्छे काम करने से रोका है. मैं नहीं जानती कि इसमें कितना समय और लगेगा, और मैं यह भी नहीं जानती कि मेरी मां घर वापस कब आएँगी, मुझे डाटेंगी, मुझे खाना खाने को कहेंगी, या मुझे्'},
 {'score': 0.004980894271284342,
  'token': 265,
  'token_str': 'े',
  'sequence': 'मैं उनसे नफरत करती हूँ जिन्होंने किसी को अच्छे काम करने से रोका है. मैं नहीं जानती कि इसमें कितना समय और लगेगा, और मैं यह भी नहीं जानती कि मेरी मां घर वापस कब आएँगी, मुझे डाटेंगी, मुझे खाना खाने को कहेंगी, या मुझेे'},
 {'score': 0.0024795588105916977,
  'token': 270,
  'token_str': 'ि',
  'sequence': 'मैं उनसे न

In [47]:
!transformers-cli add-new-model ./TokRoBERTa_BPE

usage: transformers-cli <command> [<args>]
Transformers CLI tool: error: unrecognized arguments: ./TokRoBERTa_BPE
